In [ ]:
#%pip install tensorflow
%matplotlib widget

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# Создаем искусственные данные
np.random.seed(42)
n = 50000
x = np.linspace(0, 1000, n)
trend = np.sin(x) + 0.5 * x
noise = np.random.normal(scale=0.3, size=n)
y = trend + noise  # Исходный ряд

# Применяем Savitzky-Golay для "идеального" сглаживания
window_size = 51  # Симметричное окно (смотрит в будущее)
polyorder = 3
y_smooth = savgol_filter(y, window_size, polyorder)

# Функция генерации признаков (разности скользящих средних)
def generate_features(series, windows=[2, 4, 8, 16, 32, 64, 128, 256]):
    smas = [pd.Series(series).rolling(w, min_periods=1).mean().values for w in windows]
    diffs = [smas[i] - smas[i + 1] for i in range(len(smas) - 1)]
    return np.stack(diffs, axis=-1)  # (количество точек, количество фич)

def generate_ema_features(series, windows=[2, 4, 8, 16, 32, 64, 128, 256]):
    smas = [pd.Series(series).rolling(w, min_periods=1).mean().values for w in windows]
    diffs1 = [smas[i] - smas[i + 1] for i in range(len(smas) - 1)]
    emas = [pd.Series(series).ewm(span=w, adjust=False).mean().values for w in windows]
    diffs2 = [emas[i] - emas[i + 1] for i in range(len(emas) - 1)]
    diffs = np.concatenate([diffs1, diffs2])
    return np.stack(diffs, axis=-1)

# Определяем целевой временной ряд как разницу между сглаживанием и длинной скользящей средней
sma_long = pd.Series(y).rolling(32, min_periods=1).mean().values  # Длинная средняя
Y = y_smooth - sma_long[len(sma_long) - len(y_smooth):]  # Теперь нейросеть предсказывает разницу

# Генерируем X
X = generate_ema_features(y)
X /= pd.Series(y).rolling(256, min_periods=1).mean().values.reshape(-1, 1)  # Нормируем на SMA(256)

# Разделяем на train/test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Проверяем размерность данных
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

# Создаем модель нейросети - исправляем размерность Input
model = keras.Sequential([
    layers.Input(shape=(X.shape[1],)),  # Убираем неверное измерение
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

# Обучаем модель
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test), verbose=1)

# Предсказание на тестовых данных
Y_pred = model.predict(X_test)

# Восстанавливаем предсказанное сглаживание
Y_pred_restored = Y_pred.flatten() + sma_long[len(sma_long) - len(Y_pred):]

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), y[-len(Y_test):], label="Исходный ряд", color="blue", alpha=0.6)
plt.plot(range(len(Y_test)), Y_test + sma_long[len(sma_long) - len(Y_test):], label="Настоящее сглаживание (Savgol)", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred_restored, label="Предсказанное нейросетью сглаживание", color="red", alpha=0.8)
plt.legend()
plt.title("Нейросетевой аналог Savitzky-Golay (без заглядывания в будущее)")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), y[-len(Y_test):], label="Исходный ряд", color="gray", alpha=0.6)
plt.plot(range(len(Y_test)), Y_test + sma_long[len(sma_long) - len(Y_test):], label="Настоящее сглаживание (Savgol)", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred_restored, label="Предсказанное нейросетью сглаживание", color="red", alpha=0.8)

# Добавляем скользящие средние разной длины
sma_windows = [8, 16, 32]
for w in sma_windows:
    sma = pd.Series(y).rolling(w, min_periods=1).mean().values[-len(Y_test):]
    plt.plot(range(len(Y_test)), sma, label=f"SMA {w}", linestyle="dashed", alpha=0.7)

plt.legend()
plt.title("Нейросетевой аналог Savitzky-Golay (без заглядывания в будущее) + скользящие средние")
plt.show()

# Второй график - Y_test и прогноз
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), Y_test, label="Y_test (целевое сглаживание)", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred, label="Предсказанное нейросетью", color="red", alpha=0.8)
plt.legend()
plt.title("Сравнение Y_test и прогноза нейросети")
plt.show()